# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [18]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [12]:
sales_by_customer = data.groupby(["CustomerID", "ProductName"]).sum()[["Quantity"]]
sales_by_customer

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [50]:
customer_product = data.pivot_table(values="Quantity", index="CustomerID", columns="ProductName")
customer_product.fillna(0, inplace=True)
customer_product.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [74]:
similarity_matrix = pd.DataFrame(squareform(pdist(customer_product)))
similarity_matrix

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.000000,10.723805,9.643651,9.899495,10.440307,10.148892,10.723805,10.246951,10.344080,10.148892,...,206.963765,194.766527,199.145675,183.463893,202.259734,198.075743,203.921554,197.949489,218.824130,203.798921
1,10.723805,0.000000,10.677078,10.723805,11.489125,11.045361,10.770330,10.954451,10.677078,10.488088,...,206.269242,194.285872,198.423789,182.543146,202.044550,197.729613,202.723950,197.349943,218.625250,203.953426
2,9.643651,10.677078,0.000000,10.344080,10.198039,10.488088,10.677078,10.583005,10.488088,10.488088,...,206.366179,194.517351,199.153207,182.789496,201.772149,196.817174,203.315026,197.324606,218.029814,203.806281
3,9.899495,10.723805,10.344080,0.000000,10.630146,10.535654,10.630146,9.949874,10.630146,10.440307,...,206.726873,194.643263,199.025124,182.923481,202.264678,197.575302,204.293906,197.322072,218.142155,203.681123
4,10.440307,11.489125,10.198039,10.630146,0.000000,10.677078,10.677078,10.198039,10.770330,10.392305,...,206.985507,193.889144,199.168271,182.805908,202.034650,197.339808,204.066166,197.466453,217.928887,202.960587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,198.075743,197.729613,196.817174,197.575302,197.339808,197.324606,197.339808,197.441130,197.451260,197.699267,...,266.926956,252.487623,265.753645,246.221445,251.246891,0.000000,252.487623,252.487623,278.388218,276.134025
996,203.921554,202.723950,203.315026,204.293906,204.066166,204.051464,203.821000,203.919102,203.683578,203.678668,...,273.861279,264.575131,268.095132,263.391344,256.173769,252.487623,0.000000,254.950976,276.134025,264.575131
997,197.949489,197.349943,197.324606,197.322072,197.466453,197.451260,197.466453,197.694208,197.197870,197.572771,...,266.926956,250.000000,258.602011,256.173769,261.007663,252.487623,254.950976,0.000000,266.926956,266.926956
998,218.824130,218.625250,218.029814,218.142155,217.928887,218.487986,218.501716,218.135279,218.373533,218.940631,...,287.228132,287.228132,272.717803,265.753645,279.508497,278.388218,276.134025,266.926956,0.000000,269.258240


In [75]:
# WTF?

#arr = np.arange(6).reshape(3,2)
arr = [[0, 1], [2, 3], [4, 5], [1, 1]]
squareform(pdist(arr))
print(pdist(arr))
sqrdf = pd.DataFrame(squareform(pdist(arr)), index=("a","b","c","d"), columns=("a","b","c","d"))
sqrdf

[2.82842712 5.65685425 1.         2.82842712 2.23606798 5.        ]


,a,b,c,d
a,0.000000,2.828427,5.656854,1.000000
b,2.828427,0.000000,2.828427,2.236068
c,5.656854,2.828427,0.000000,5.000000
d,1.000000,2.236068,5.000000,0.000000


In [66]:
test = pd.DataFrame([[1.0,2.0,3.0],[4,5,6],[7,8,9]], columns=["a","b","c"])
test

,a,b,c
0,1.0,2.0,3.0
1,4.0,5.0,6.0
2,7.0,8.0,9.0


In [48]:
pdist(test)

array([ 5.19615242, 10.39230485,  5.19615242])

In [53]:
squareform(pdist(test))

array([[ 0.        ,  5.19615242, 10.39230485],
       [ 5.19615242,  0.        ,  5.19615242],
       [10.39230485,  5.19615242,  0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.